<a href="https://colab.research.google.com/github/Mic-Bin/Texas-HS-Maps-OnRamps-/blob/main/OnRamps_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime
import requests

## Import text file

In [ ]:
# Open the text file in read mode
with open('/content/OnRamps txt.txt', 'r') as file:
    lines = file.readlines()  # Read all lines into a list
l = []

In [ ]:
# Separate lines into every_second_line and other_lines lists
for i, line in enumerate(lines):
    if i % 2 == 1:  # Check if the line number is odd (second line)
        l.append(line.strip())  # Append the line to every_second_line list

In [ ]:
# Remove empty lines from data
filtered_list = [string for string in lines if string not in ['\n']]

In [ ]:
# Remove extra characters ('\n)
f_list = [x.replace('\n', "") for x in filtered_list]

In [ ]:
# Remove extra characters from first line
f_list[0] = f_list[0][1:]

# Import new text file

In [ ]:
# List to store every high school name and ISD
names = []
isds = []

In [ ]:
# Separate lines into every_second_line and other_lines lists
for i, line in enumerate(f_list):
    if i % 2 == 1:  # Check if the line number is odd (second line)
        isds.append(line.strip())  # Append the line to every_second_line list
    else:
        names.append(line.strip())        # Append the line to other_lines list

In [ ]:
# Replace HS with High School
names = [x.replace(' HS', " High School") for x in names]

In [ ]:
# Convert the two list into a dataframe
df = pd.DataFrame({names[0]: names[1:], isds[0]: isds[1:]})

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.head()

,Campus,District
0,Aldine High School,Aldine ISD
1,Blanson CTE High School,Aldine ISD
2,Carver High School,Aldine ISD
3,Davis High School,Aldine ISD
4,Eisenhower High School,Aldine ISD


## Find the addresses of the schools

In [ ]:
search_ready = df.apply(lambda x: ' '.join(x.tolist()), axis=1)

In [ ]:
!pip install -U googlemaps

In [ ]:
# Load API key
from google.colab import userdata
api_key = userdata.get('GoogleKey')

In [ ]:
gmaps = googlemaps.Client(key=api_key)

In [ ]:
def find_address(place_name):
    places_result = gmaps.places(query=place_name)
    # Check if results are found
    if places_result['status'] == 'OK':
        # Get the place ID of the first result
        place_id = places_result['results'][0]['place_id']
        # Use the place ID to get details including the formatted address
        place_details = gmaps.place(place_id)
        formatted_address = place_details['result']['formatted_address']
        return formatted_address
    else:
        print("No results found for", place_name)
        return None

In [ ]:
place_name = search_ready[0]
address = find_address(place_name)
if address:
    print(f"The address of {place_name} is: {address}")

The address of Aldine High School Aldine ISD is: 11101 Airline Dr, Houston, TX 77037, USA


In [ ]:
adds = search_ready.apply(lambda x: find_address(x))

In [ ]:
df['Address'] = adds

In [ ]:
df.reset_index()

,index,Campus,District,Address
0,0,Aldine High School,Aldine ISD,"11101 Airline Dr, Houston, TX 77037, USA"
1,1,Blanson CTE High School,Aldine ISD,"311 West Road, Houston, TX 77038, USA"
2,2,Carver High School,Aldine ISD,"2100 S Victory St, Houston, TX 77088, USA"
3,3,Davis High School,Aldine ISD,"12525 Ella Blvd, Houston, TX 77067, USA"
4,4,Eisenhower High School,Aldine ISD,"7922 Antoine Dr, Houston, TX 77088, USA"
...,...,...,...,...
217,235,Thrall High School,Thrall ISD,"601 S Bounds St, Thrall, TX 76578, USA"
218,236,Trivium Academy,Trivium Academy,"2205 E Hebron Pkwy, Carrollton, TX 75010, USA"
219,237,Vidor High School,Vidor ISD,"500 Orange St, Vidor, TX 77662, USA"
220,238,Brewer High School,White Settlement ISD,"1025 W Loop 820 N, Fort Worth, TX 76108, USA"


In [ ]:
df.head()

,Campus,District,Address
0,Aldine High School,Aldine ISD,"11101 Airline Dr, Houston, TX 77037, USA"
1,Blanson CTE High School,Aldine ISD,"311 West Road, Houston, TX 77038, USA"
2,Carver High School,Aldine ISD,"2100 S Victory St, Houston, TX 77088, USA"
3,Davis High School,Aldine ISD,"12525 Ella Blvd, Houston, TX 77067, USA"
4,Eisenhower High School,Aldine ISD,"7922 Antoine Dr, Houston, TX 77088, USA"


In [ ]:
df.tail()

,Campus,District,Address
235,Thrall High School,Thrall ISD,"601 S Bounds St, Thrall, TX 76578, USA"
236,Trivium Academy,Trivium Academy,"2205 E Hebron Pkwy, Carrollton, TX 75010, USA"
237,Vidor High School,Vidor ISD,"500 Orange St, Vidor, TX 77662, USA"
238,Brewer High School,White Settlement ISD,"1025 W Loop 820 N, Fort Worth, TX 76108, USA"
239,Wimberley High School,Wimberley ISD,"100 Carney Ln, Wimberley, TX 78676, USA"


In [ ]:
df.to_csv('OnRampsAddys.csv')